In [14]:
# import relevant modules
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# reflections: I think I wasn't familiar enough with pandas, so I ended up trying to manually do what pandas already has (see code in ''' all the way at the bottom). Try to avoid loops as much as possible; it's likely to be built in already.

# try as much as possible to look for the relevant function in pandas, it's usually already done
# test functions on a smaller dataset so that they don't take as long

# output: csv with headers shopid, userid
# >= 1 suspicious user in numerical order userid + "&" + userid
# no suspicious user return 0

# code summary:
# 1. sort data into a nice order
# 2. make initial output file with everything 0
# 3. find suspicious cases
# 4. write results to output file

# version check
pd.__version__

In [8]:
def open_data(file):

    # get csv file
    inputs = pd.read_csv(file)

    # get data types (especially for time)
    # print(inputs.dtypes)

    return inputs

In [ ]:
# test function
open_data(f"..\data\order_brush_order.csv")

In [9]:
# sort shop by shopid and time and reassign index
# we don't need the orderid, so make each order into 1 to calculate easily
def sort_shop():
    sorted = open_data(f'..\data\order_brush_order.csv').sort_values(by=['shopid','event_time'],
                                                                    ignore_index = True)
    sorted['orderid'] = 1
    
    # to look at the data, like a save point
    return sorted

sort_shop()#.to_csv('sorted_data.csv')

,orderid,shopid,userid,event_time
0,1,10009,196962305,2019-12-27 03:06:50
1,1,10051,2854032,2019-12-27 19:16:11
2,1,10051,48600461,2019-12-29 01:56:19
3,1,10061,168750452,2019-12-28 09:27:55
4,1,10061,194819216,2019-12-28 12:05:32
...,...,...,...,...
222745,1,214949521,46269178,2019-12-31 20:06:43
222746,1,214964814,200983383,2019-12-29 22:26:16
222747,1,215175775,13688804,2019-12-31 09:06:31
222748,1,215175775,129266028,2019-12-31 14:14:37


In [46]:
def one_shop(shopid):
    
    data = sort_shop()
    data = data[data['shopid'] == shopid]

    return data

def unique_shops():
    all_shopids = pd.unique(sort_shop()['shopid'])

    return all_shopids

unique_shops()

10009

In [ ]:
# just start the final csv data as shopid and userid, with all values = 0
def initialise_output():

    # get unique shopids and make dataframe
    all_shopids = unique_shops()
    shopid_and_userid = pd.DataFrame(all_shopids, columns = ['shopid'])

    #set userid column to 0 and add to df
    shopid_and_userid['userid'] = 0

    return shopid_and_userid

initialise_output().to_csv('output.csv')

In [226]:
# rolling
def rolling_orders():
    data = sort_shop()
    data['event_time'] = pd.to_datetime(data['event_time'])

    data_test = data#.head(200)

    # works
    data_test = data_test.groupby('shopid')['event_time', 'orderid'].rolling('1h', on = 'event_time').sum()
    
    # data = data.head(200)
    
    # data = data.set_index(['shopid', 'event_time'])

    # data_test = data_test.groupby(['shopid', 'event_time'])['orderid']

    # change data_test tuple into integer (for testing only)
    # n = 1
    # data['rolling_orders'] = [int(x[n]) for x in data_test]

    return data_test

rolling_orders().to_csv('roll_test_full.csv')

In [35]:
def count_unique_users():
    # inspired by https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series

    data = sort_shop()
    data['event_time'] = pd.to_datetime(data['event_time'])

    data_test = data#.head(200)

    data_test = data_test.groupby('shopid')['event_time', 'userid'].rolling('1h', on = 'event_time')
    
    data_test = data_test.apply(lambda arr: pd.Series(arr).nunique())['userid'].values

    data['rolling_users'] = data_test

    return data

count_unique_users().to_csv('roll_users.csv')

In [72]:
# combine rolled users and rolled orders, calculate concentration

def combine_all():
    roll_orders = pd.read_csv('roll_test_full.csv').drop(['Unnamed: 1'], axis = 1)
    roll_users = pd.read_csv('roll_users.csv')

    data = roll_orders
    data['rolling_users'] = roll_users['rolling_users']

    data['userid'] = sort_shop()['userid'].astype('int64')

    data.rename(
        columns = {'orderid':'rolling_orders'},
        inplace = True
        )

    data['conc'] = (data['rolling_orders'] / data['rolling_users']).astype('int64')

    return data

combine_all()

,shopid,event_time,rolling_orders,rolling_users,userid,conc
0,10009,2019-12-27 03:06:50,1.0,1.0,196962305,1
1,10051,2019-12-27 19:16:11,1.0,1.0,2854032,1
2,10051,2019-12-29 01:56:19,1.0,1.0,48600461,1
3,10061,2019-12-28 09:27:55,1.0,1.0,168750452,1
4,10061,2019-12-28 12:05:32,1.0,1.0,194819216,1
...,...,...,...,...,...,...
222745,214949521,2019-12-31 20:06:43,2.0,1.0,46269178,2
222746,214964814,2019-12-29 22:26:16,1.0,1.0,200983383,1
222747,215175775,2019-12-31 09:06:31,1.0,1.0,13688804,1
222748,215175775,2019-12-31 14:14:37,1.0,1.0,129266028,1


In [76]:
def find_offenders():

    data = combine_all()
    data_conc = data[data['conc'] >= 3]

    # for consecutive indices, only take the highest value

    return data_conc

find_offenders()

,shopid,event_time,rolling_orders,rolling_users,userid,conc
1175,10402,2019-12-28 14:30:26,3.0,1.0,77819,3
1358,10536,2019-12-27 11:44:08,3.0,1.0,672345,3
1361,10536,2019-12-27 17:55:26,3.0,1.0,672345,3
2072,42472,2019-12-27 22:38:38,3.0,1.0,740844,3
2134,42818,2019-12-31 10:57:26,3.0,1.0,170385453,3
...,...,...,...,...,...,...
221164,204225676,2019-12-31 00:19:37,5.0,1.0,198662175,5
221716,208696908,2019-12-30 17:42:06,3.0,1.0,214111334,3
222358,210197928,2019-12-27 07:48:54,3.0,1.0,52867898,3
222359,210197928,2019-12-27 07:49:56,4.0,1.0,52867898,4
